<a href="https://colab.research.google.com/github/brendanpshea/computing_concepts_python/blob/main/IntroCS_09_Data_and_Databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Data to Knowledge: Understanding Databases
### Computing Concepts | Brendan Shea, PhD

Imagine you're Master Yoda, overseeing the training of hundreds of young Padawans at the Jedi Academy. Each student has different strengths, comes from a different planet, trains under different masters, and progresses at their own pace. How do you keep track of all this information? How do you turn all these individual facts into useful knowledge that helps you run the academy effectively? This is where **databases** come in.

A **database** is an organized collection of related information stored and accessed electronically. But it's much more than just a digital filing cabinet. Think of it as a living library that not only stores data but helps us understand relationships between different pieces of information. When Master Yoda wants to know which Padawans are ready for their trials, or which teaching methods are most effective for different species, he's not just looking up individual facts – he's discovering patterns and insights by connecting different pieces of information.

Let's break down the journey from raw data to actionable knowledge:

**Data** represents individual facts. For example: "Ahsoka Tano is 14 years old," "Ahsoka Tano's master is Anakin Skywalker," or "Ahsoka Tano scored 95% in lightsaber training." By themselves, these are just isolated pieces of information.

When we organize this data in a structured way, we create **information**. We might group all of Ahsoka's training scores together, or list all students under Anakin's guidance. This organization helps us see these facts in context.

When we analyze this information to understand patterns and relationships, we gain **knowledge**. We might discover that Padawans trained by Anakin tend to excel in combat skills but need more guidance in diplomatic training, or that students from certain planets typically require additional meditation practice to master Force control.

A well-designed database helps us move through these levels by:
- Storing data reliably and efficiently
- Maintaining data integrity (ensuring information stays accurate and consistent)
- Allowing quick retrieval of specific information
- Supporting complex queries that help us discover patterns
- Enabling multiple users to access and update information simultaneously
- Protecting sensitive information through security measures

In the modern galaxy, databases are everywhere. When you check your holographic messages, look up starship schedules, or access the Jedi Archives, you're interacting with databases. For the Jedi Academy, a database doesn't just track students and classes – it helps Masters make better decisions about training programs, identify students who might need extra support, and ensure that the next generation of Jedi receives the best possible training.

As we continue exploring databases, we'll learn how to design them effectively, understand different ways to model our data, and master the art of querying – asking the right questions to get the insights we need. We'll use **SQLite**, a lightweight but powerful database system, to put these concepts into practice. Just as a Jedi learns to feel the Force flowing through all things, you'll learn to see the connections in your data and use them to gain deeper understanding.

## Brendan's Lecture

In [1]:
## Video Lecture
from IPython.display import YouTubeVideo
YouTubeVideo('n06F-krmXL8', width=800, height=500)

# Designing Your Database: The Entity-Relationship Model

Before we start storing information about our Jedi Academy, we need a plan – much like an architect needs blueprints before building a temple. In database design, we use the **Entity-Relationship (ER) model** to create these blueprints. This model helps us identify what information we need to store and how different pieces of information relate to each other.

## Understanding Entities and Relationships

An **entity** is a distinct object or concept about which we want to store information. In our Jedi Academy, some entities might be students, masters, and courses. Each entity has **attributes** – specific pieces of information we want to track about that entity. For example, a student entity might have attributes like name, age, and home planet.

A **relationship** describes how entities are connected to each other. Relationships have two important characteristics:

1. **Cardinality**: How many entities can participate in each side of the relationship:
   - **One-to-One (1:1)**: Each entity relates to exactly one other entity. For example, each Padawan might have one assigned lightsaber crystal.
   - **One-to-Many (1:N)**: One entity can relate to multiple other entities, but those entities each relate to only one entity. For example, a Jedi Master can train many Padawans.
   - **Many-to-Many (M:N)**: Entities on both sides can relate to multiple entities on the other side. For example, a student can enroll in many courses, and each course can have many students.

2. **Optionality**: Whether the relationship is required:
   - **Mandatory**: The relationship must exist. For example, every student must be enrolled in at least one course.
   - **Optional**: The relationship may exist. For example, a master might or might not be currently teaching any courses.

## Crow's Foot Notation
In database diagrams, we show cardinality and optionality using "crow's foot notation":
```
One         Only one     ──┤
Many        Zero or more ──<
Optional    Zero or one  ──O
Mandatory   One or more  ─║<
```

## A Simple ER Diagram

Here's a basic ER diagram for our Jedi Academy:

In [ ]:
# @title
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
erDiagram
STUDENT }|--o{ COURSE : "enrolls in"
STUDENT }o--|| MASTER : "is trained by"

STUDENT {
    name string
    age int
    home_planet string
}

COURSE {
    name string
    level int
    description string
}

MASTER {
    name string
    rank string
    specialization string
}
""")

## Reading the Diagram

Let's break down each relationship in our diagram:

1. **STUDENT to COURSE** (}|--o{):
   - A student must be enrolled in at least one course (mandatory, shown by the vertical line |)
   - A student can enroll in many courses (crow's foot <)
   - A course can have zero or many students (optional, shown by the circle o, plus crow's foot)

2. **STUDENT to MASTER** (}o--||):
   - A student may or may not have a master (optional, shown by the circle o)
   - A student can have at most one master (single line -)
   - A master must have at least one student (mandatory, shown by the double line ||)

## The Many-to-Many Challenge

The many-to-many relationship between STUDENT and COURSE presents a special challenge when we implement this in a relational database. We can't directly connect students to courses because there's no way to represent multiple connections in a single table. Instead, we'll need to create a separate ENROLLMENT table that connects students to courses and can also store relationship-specific attributes like enrollment date and grade.

In the next section, we'll see how to transform this conceptual model into actual database tables, including how to handle the many-to-many relationship using an intermediary table.

When we move on to the relational model in the next section, we'll discover that this model can't handle many-to-many relationships. So, we'll need an entity ("Enrollment") that sits between Course and Student.

# Understanding the Relational Model: Organizing Data in Tables

Now that we have our blueprint in the ER model, we need to understand how databases actually store this information. Most modern databases use the **relational model**, which organizes data into tables (also called **relations**). Think of each table as a spreadsheet with strict rules about how data can be stored and connected.

## The Building Blocks of Tables

Let's break down the key components using our STUDENT table as an example:

| student_id | name | age | home_planet | midichlorian_count |
|------------|------|-----|-------------|-------------------|
| 1          | Luke | 19  | Tatooine    | 14500            |
| 2          | Leia | 19  | Alderaan    | 13900            |
| 3          | Rey  | 20  | Jakku       | 15000            |

A table consists of:

- **Columns** (or **attributes**): The vertical categories that define what type of data we're storing. Each column has a name and a specific **data type** that determines what kind of values it can contain. In our example, 'student_id' is a number, 'name' is text, and 'age' is a number.

- **Rows** (or **tuples**): Each horizontal line represents one complete record. A row contains one value for each column. Each row in our STUDENT table represents one Padawan learner.

- **Primary Key**: Every table needs a way to uniquely identify each row. The **primary key** is a column (or combination of columns) that must contain unique values. In our STUDENT table, 'student_id' is the primary key – no two students can have the same ID.

## Connecting Tables with Foreign Keys

Tables don't exist in isolation. We need ways to represent the relationships we designed in our ER model. This is where **foreign keys** come in. A foreign key is a column that refers to the primary key of another table. Let's look at the ENROLLMENT table:

| enrollment_id | student_id | course_id | start_date | grade |
|--------------|------------|-----------|------------|-------|
| 1            | 1          | 101       | 2024-01-15 | 95.0  |
| 2            | 1          | 102       | 2024-01-15 | 88.5  |
| 3            | 2          | 101       | 2024-01-15 | 92.0  |

Here, 'student_id' is a foreign key that references the STUDENT table, and 'course_id' is a foreign key that references the COURSE table. These connections allow us to:
- Maintain data integrity (we can't enroll a student that doesn't exist)
- Join information from multiple tables (we can connect student names with their course grades)
- Efficiently organize data without repetition


## Rules and Constraints

The relational model enforces several important rules to maintain data quality:

1. **Entity Integrity**: Primary key values cannot be null (empty) and must be unique. This ensures each record can be uniquely identified.

2. **Referential Integrity**: Foreign key values must match a primary key value in the referenced table or be null. This prevents "orphaned" records – for example, an enrollment record for a student that doesn't exist.

3. **Domain Integrity**: Column values must match their defined data type and any additional constraints. For example, if we define 'age' as a positive integer, we can't store negative numbers or text in that column.

## Common Data Types

In SQLite, we'll work with these basic data types:
- **INTEGER**: Whole numbers (student_id, age)
- **TEXT**: Character strings (name, home_planet)
- **REAL**: Decimal numbers (grade, midichlorian_count)
- **DATE**: Calendar dates (start_date)
- **NULL**: Represents missing or undefined data

## Benefits of the Relational Model

This structured approach to storing data offers several advantages:
- **Data Independence**: We can change how data is stored without changing how applications access it
- **Data Integrity**: Rules and constraints help maintain accurate data
- **Flexible Queries**: We can combine and analyze data in many different ways
- **Concurrent Access**: Multiple users can safely access and modify data simultaneously
- **Data Consistency**: Each piece of information is stored in one place, reducing redundancy

In the next section, we'll learn SQL, the language used to create, modify, and query data in relational databases. We'll see how these concepts come to life in actual database operations.

# SQL: Communicating with Your Database

**SQL** (Structured Query Language) is the standard language for working with relational databases. Think of it as the language you use to communicate with your database. Just as Jedi use the Force to interact with the world around them, developers use SQL to interact with their databases.

Every SQL command has a specific purpose, and we can group these commands into several categories:
- Commands for defining data structures (CREATE, ALTER, DROP)
- Commands for manipulating data (INSERT, UPDATE, DELETE)
- Commands for retrieving data (SELECT)

Today, we'll start with one of the most fundamental tasks: creating tables.

## Creating Tables with SQL

Before we can store any data, we need to create tables with clearly defined structures. In SQL, we use the **CREATE TABLE** statement for this purpose. Let's look at a basic example:


In [ ]:
# Run this cell to load our database
!pip install jupysql -q
%reload_ext sql
%sql sqlite:///jedi_academy.db
%config SqlMagic.autopandas=True

# turn on foreign keys
%sql PRAGMA foreign_keys=ON

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.2 MB/s eta 0:00:00


Connecting to 'sqlite:///jedi_academy.db'

Running query in 'sqlite:///jedi_academy.db'

""


In [ ]:
%%sql
DROP TABLE IF EXISTS student;
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    home_planet TEXT,
    midichlorian_count INTEGER
);

Running query in 'sqlite:///jedi_academy.db'

""


Let's break down each part of this statement:

1. `CREATE TABLE student` - This tells SQLite we want to create a new table named "student"

2. Inside the parentheses, we list each column with its data type:
   - `student_id INTEGER PRIMARY KEY` - Creates a unique identifier for each student
   - `name TEXT NOT NULL` - Stores the student's name, and it can't be empty
   - `age INTEGER` - Stores the student's age as a whole number
   - `home_planet TEXT` - Stores the name of the student's home planet
   - `midichlorian_count INTEGER` - Stores the student's midichlorian count as a whole number

3. The **PRIMARY KEY** constraint on student_id ensures that:
   - Each student has a unique identifier
   - The value cannot be NULL
   - SQLite will automatically generate increasing values for this column

4. The **NOT NULL** constraint on name ensures that every student must have a name

In SQLite, we have several basic data types to choose from:
- **INTEGER**: Whole numbers
- **TEXT**: Any text or string data
- **REAL**: Decimal numbers
- **DATE**: Calendar dates (stored as TEXT in SQLite)
- **BLOB**: Binary data (like images or files)


Let's create the remaining tables for our database. Remember our ER diagram showed four main entities: STUDENT (which we already created), MASTER, COURSE, and ENROLLMENT. Let's create each one and understand how they work together.

## The MASTER Table

First, let's create a table for our Jedi Masters:

In [ ]:
%%sql
DROP TABLE IF EXISTS master;
CREATE TABLE master (
    master_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    specialization TEXT
);

Running query in 'sqlite:///jedi_academy.db'

""


This table stores information about each Jedi Master who teaches at the academy:
- `master_id` uniquely identifies each master
- `rank` could be "Knight", "Master", or "Council Member"
- `specialization` indicates their area of expertise (like "Lightsaber Combat" or "Force Healing")

## The COURSE Table

Next, let's create a table for the courses offered at the academy:

In [ ]:
%%sql
DROP TABLE IF EXISTS course;
CREATE TABLE course (
    course_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    level TEXT NOT NULL,
    description TEXT,
    master_id INTEGER,
    FOREIGN KEY (master_id) REFERENCES master (master_id)
);

Running query in 'sqlite:///jedi_academy.db'

""


Notice something new here: the **FOREIGN KEY** constraint. This creates a connection to the master table, indicating which master teaches each course. The constraint ensures that:
- Every `master_id` in the course table must exist in the master table
- We can't assign a course to a non-existent master
- If we try to delete a master who is teaching courses, SQLite will prevent us from breaking this relationship

## The ENROLLMENT Table

Finally, let's create the enrollment table that tracks which students are taking which courses:


In [ ]:
%%sql
DROP TABLE IF EXISTS enrollment;
CREATE TABLE enrollment (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER NOT NULL,
    course_id INTEGER NOT NULL,
    start_date TEXT NOT NULL,
    grade REAL,
    FOREIGN KEY (student_id) REFERENCES student (student_id),
    FOREIGN KEY (course_id) REFERENCES course (course_id)
);

Running query in 'sqlite:///jedi_academy.db'

""


This table is special because it:
- Creates the many-to-many relationship between students and courses
- Has TWO foreign keys, connecting to both the student and course tables
- Includes additional information about the enrollment (start date and grade)
- Uses REAL data type for grade to allow decimal points (like 95.5)

## Checking Our Work

Now we have all four tables created:
1. student (Padawan learners)
2. master (Jedi Masters)
3. course (Training programs)
4. enrollment (Who's taking what)

We can verify our table structure using a special SQLite command:

In [ ]:
%%sql
-- Get table schemas
SELECT *
FROM sqlite_master
WHERE type='table';

Running query in 'sqlite:///jedi_academy.db'

,type,name,tbl_name,rootpage,sql
0,table,student,student,2,CREATE TABLE student (\n student_id INTEGER...
1,table,master,master,3,CREATE TABLE master (\n master_id INTEGER P...
2,table,course,course,4,CREATE TABLE course (\n course_id INTEGER P...
3,table,enrollment,enrollment,5,CREATE TABLE enrollment (\n enrollment_id I...


# Adding Data to Our Database

Now that we have our tables created, let's learn how to add data to them. In SQL, we use the **INSERT** statement to add new records to a table. When adding data across related tables, we need to be careful about the order of our insertions to maintain our relationships correctly.

## Basic INSERT Syntax

The basic syntax for INSERT has two parts:
1. Specify which columns you're filling
2. Provide the values for those columns

```sql
INSERT INTO table_name (column1, column2, column3)
VALUES (value1, value2, value3);
```

## Adding Masters

Let's start by adding some Jedi Masters to our master table:

In [ ]:
%%sql
INSERT INTO master (name, rank, specialization)
VALUES ('Yoda', 'Council Member', 'Force Training');

INSERT INTO master (name, rank, specialization)
VALUES ('Obi-Wan Kenobi', 'Master', 'Lightsaber Combat');

INSERT INTO master (name, rank, specialization)
VALUES ('Mace Windu', 'Council Member', 'Vaapad Combat');

Running query in 'sqlite:///jedi_academy.db'

1 rows affected.

1 rows affected.

1 rows affected.

""


Note that we don't specify `master_id` because it's an auto-incrementing primary key - SQLite handles this for us automatically.

## Adding Courses

Now that we have masters, we can add courses. Remember that `master_id` in the course table is a foreign key, so we need to use the correct IDs that were generated for our masters:

In [ ]:
%%sql
INSERT INTO course (name, level, description, master_id)
VALUES (
    'Foundations of the Force',
    'Beginner',
    'Introduction to Force sensitivity and basic control',
    1  -- Master Yoda's ID
);

INSERT INTO course (name, level, description, master_id)
VALUES (
    'Basic Lightsaber Training',
    'Beginner',
    'Fundamental lightsaber forms and safety',
    2  -- Obi-Wan's ID
);

INSERT INTO course (name, level, description, master_id)
VALUES (
    'Advanced Combat Forms',
    'Advanced',
    'Advanced lightsaber combat techniques',
    3  -- Mace Windu's ID
);

Running query in 'sqlite:///jedi_academy.db'

1 rows affected.

1 rows affected.

1 rows affected.

""


## Adding Students

Let's add some students to our student table:


In [ ]:
%%sql
INSERT INTO student (name, age, home_planet, midichlorian_count)
VALUES ('Luke Skywalker', 19, 'Tatooine', 14500);

INSERT INTO student (name, age, home_planet, midichlorian_count)
VALUES ('Leia Organa', 19, 'Alderaan', 13900);

INSERT INTO student (name, age, home_planet, midichlorian_count)
VALUES ('Ben Solo', 20, 'Chandrila', 14800);

Running query in 'sqlite:///jedi_academy.db'

1 rows affected.

1 rows affected.

1 rows affected.

""


## Adding Enrollments

Finally, we can enroll students in courses. This requires knowing both the student_id and course_id values:

In [ ]:
%%sql
INSERT INTO enrollment (student_id, course_id, start_date, grade)
VALUES (
    1,  -- Luke's ID
    1,  -- Foundations of the Force
    '2024-01-15',
    95.5
);

INSERT INTO enrollment (student_id, course_id, start_date, grade)
VALUES (
    1,  -- Luke's ID
    2,  -- Basic Lightsaber Training
    '2024-01-15',
    88.0
);

Running query in 'sqlite:///jedi_academy.db'

1 rows affected.

1 rows affected.

""


## Error Prevention

The foreign key constraints we set up will prevent common mistakes. For example, these insertions would fail:

In [ ]:
%%sql
-- This fails because master_id 99 doesn't exist
-- INSERT INTO course (name, level, description, master_id)
-- VALUES ('Force Healing', 'Advanced', 'Healing techniques', 99);

-- This fails because student_id 50 doesn't exist
-- INSERT INTO enrollment (student_id, course_id, start_date)
-- VALUES (50, 1, '2024-01-15');

Running query in 'sqlite:///jedi_academy.db'

""



## A Shortcut for Multiple Insertions

When adding multiple records, we can use a single INSERT statement:

In [ ]:
%%sql
-- Add more masters
INSERT INTO master (name, rank, specialization)
VALUES
    ('Plo Koon', 'Council Member', 'Force Judgment'),
    ('Shaak Ti', 'Council Member', 'Force Healing'),
    ('Kit Fisto', 'Master', 'Underwater Combat'),
    ('Luminara Unduli', 'Master', 'Force Push Mastery'),
    ('Quinlan Vos', 'Knight', 'Force Psychometry');

-- Add more courses
INSERT INTO course (name, level, description, master_id)
VALUES
    ('Meditation Techniques', 'Beginner', 'Basic meditation and Force connection', 1),  -- Yoda
    ('Advanced Force Control', 'Advanced', 'Mastery of Force push, pull, and lift', 7), -- Luminara
    ('Force Healing Basics', 'Intermediate', 'Introduction to Force healing techniques', 5), -- Shaak Ti
    ('Underwater Combat Training', 'Intermediate', 'Combat techniques for aquatic environments', 6), -- Kit Fisto
    ('Stealth and Infiltration', 'Advanced', 'Advanced stealth techniques using the Force', 8), -- Quinlan Vos
    ('Diplomatic Relations', 'Intermediate', 'Negotiation and conflict resolution', 2), -- Obi-Wan
    ('Force Precognition', 'Advanced', 'Development of precognitive abilities', 1); -- Yoda

-- Add more students
INSERT INTO student (name, age, home_planet, midichlorian_count)
VALUES
    ('Barriss Offee', 18, 'Mirial', 13200),
    ('Caleb Dume', 14, 'Coruscant', 13800),
    ('Petro', 12, 'Corellia', 12500),
    ('Katooni', 11, 'Tholoth', 12300),
    ('Gungi', 12, 'Kashyyyk', 13100),
    ('Zatt', 11, 'Mon Cala', 12200),
    ('Byph', 11, 'Ithor', 12100),
    ('Ganodi', 10, 'Rodia', 12400);

-- Add more enrollments
INSERT INTO enrollment (student_id, course_id, start_date, grade)
VALUES
    -- Luke's additional courses
    (1, 3, '2024-01-15', 91.5),  -- Advanced Combat Forms
    (1, 4, '2024-01-15', 89.0),  -- Meditation Techniques

    -- Leia's courses
    (2, 6, '2024-01-15', 98.5),  -- Diplomatic Relations
    (2, 4, '2024-01-15', 94.0),  -- Meditation Techniques
    (2, 1, '2024-01-15', 92.5),  -- Foundations of the Force

    -- Ben Solo's courses
    (3, 1, '2024-01-15', 96.0),  -- Foundations of the Force
    (3, 5, '2024-01-15', 88.5),  -- Advanced Force Control
    (3, 9, '2024-01-15', NULL),  -- Force Precognition

    -- Ahsoka's courses
    (6, 1, '2024-01-15', 97.0),  -- Foundations of the Force
    (6, 2, '2024-01-15', 96.5),  -- Basic Lightsaber Training
    (6, 6, '2024-01-15', 93.0),  -- Diplomatic Relations

    -- Other students
    (7, 1, '2024-01-15', 88.0),  -- Barriss - Foundations
    (7, 6, '2024-01-15', NULL),  -- Barriss - Diplomatic Relations
    (7, 7, '2024-01-15', 92.0),  -- Barriss - Force Healing

    (8, 1, '2024-01-15', 90.5),  -- Caleb - Foundations
    (8, 2, '2024-01-15', 93.5),  -- Caleb - Basic Lightsaber

    (9, 1, '2024-01-15', 87.5),  -- Petro - Foundations
    (9, 4, '2024-01-15', 86.0),  -- Petro - Meditation

    (10, 1, '2024-01-15', 89.5), -- Katooni - Foundations
    (10, 7, '2024-01-15', 91.0); -- Katooni - Force Healing


Running query in 'sqlite:///jedi_academy.db'

5 rows affected.

7 rows affected.

8 rows affected.

20 rows affected.

""


Next, we'll learn how to retrieve this data using SELECT statements, starting with basic queries and working our way up to more complex ones.

# Retrieving Data: Basic SELECT Queries

Now that our database contains data, we can start retrieving and analyzing it. The **SELECT** statement is your primary tool for querying data in SQL. Let's start with the basics and gradually build up to more complex queries.

## The Basic SELECT Statement

The simplest SELECT statement has two main parts:
- **SELECT**: specifies which columns you want
- **FROM**: specifies which table you're querying

In [ ]:
%%sql
SELECT name, age, home_planet
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,name,age,home_planet
0,Luke Skywalker,19,Tatooine
1,Leia Organa,19,Alderaan
2,Ben Solo,20,Chandrila
3,Barriss Offee,18,Mirial
4,Caleb Dume,14,Coruscant
5,Petro,12,Corellia
6,Katooni,11,Tholoth
7,Gungi,12,Kashyyyk
8,Zatt,11,Mon Cala
9,Byph,11,Ithor


To select all columns, you can use the asterisk (*):

In [ ]:
%%sql
SELECT *
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,student_id,name,age,home_planet,midichlorian_count
0,1,Luke Skywalker,19,Tatooine,14500
1,2,Leia Organa,19,Alderaan,13900
2,3,Ben Solo,20,Chandrila,14800
3,4,Barriss Offee,18,Mirial,13200
4,5,Caleb Dume,14,Coruscant,13800
5,6,Petro,12,Corellia,12500
6,7,Katooni,11,Tholoth,12300
7,8,Gungi,12,Kashyyyk,13100
8,9,Zatt,11,Mon Cala,12200
9,10,Byph,11,Ithor,12100


This returns all columns (student_id, name, age, home_planet, midichlorian_count).

## Filtering with WHERE

The **WHERE** clause lets you filter your results. Think of it as asking a question about each row in the table:


In [ ]:
%%sql
SELECT name, age, midichlorian_count
FROM student
WHERE age < 18;

Running query in 'sqlite:///jedi_academy.db'

,name,age,midichlorian_count
0,Caleb Dume,14,13800
1,Petro,12,12500
2,Katooni,11,12300
3,Gungi,12,13100
4,Zatt,11,12200
5,Byph,11,12100
6,Ganodi,10,12400


This shows us our younger students.

You can use various comparison operators:
- Equal to: `=`
- Not equal to: `!=` or `<>`
- Greater than: `>`
- Less than: `<`
- Greater than or equal to: `>=`
- Less than or equal to: `<=`

For example, to find students from a specific planet:

In [ ]:
%%sql
SELECT name, midichlorian_count
FROM student
WHERE home_planet = 'Tatooine';

Running query in 'sqlite:///jedi_academy.db'

,name,midichlorian_count
0,Luke Skywalker,14500


## Limiting Results

The **LIMIT** clause controls how many rows are returned:

In [ ]:
%%sql
SELECT name, midichlorian_count
FROM student
ORDER BY midichlorian_count DESC
LIMIT 3;

Running query in 'sqlite:///jedi_academy.db'

,name,midichlorian_count
0,Ben Solo,14800
1,Luke Skywalker,14500
2,Leia Organa,13900


his shows us the three students with the highest midichlorian counts.

## Sorting Results

The **ORDER BY** clause sorts your results.


In [ ]:
%%sql
# sort students oldest to youngest
SELECT name, age, home_planet
FROM student
ORDER BY age DESC;

Running query in 'sqlite:///jedi_academy.db'

,name,age,home_planet
0,Ben Solo,20,Chandrila
1,Luke Skywalker,19,Tatooine
2,Leia Organa,19,Alderaan
3,Barriss Offee,18,Mirial
4,Caleb Dume,14,Coruscant
5,Petro,12,Corellia
6,Gungi,12,Kashyyyk
7,Katooni,11,Tholoth
8,Zatt,11,Mon Cala
9,Byph,11,Ithor


You can sort by multiple columns:

In [ ]:
%%sql
SELECT name, age, home_planet
FROM student
ORDER BY age DESC, name ASC;

Running query in 'sqlite:///jedi_academy.db'

,name,age,home_planet
0,Ben Solo,20,Chandrila
1,Leia Organa,19,Alderaan
2,Luke Skywalker,19,Tatooine
3,Barriss Offee,18,Mirial
4,Caleb Dume,14,Coruscant
5,Gungi,12,Kashyyyk
6,Petro,12,Corellia
7,Byph,11,Ithor
8,Katooni,11,Tholoth
9,Zatt,11,Mon Cala


This sorts by age (descending) and then by name (ascending) when ages are equal.

# Advanced Filtering: Combining Conditions

Sometimes we need to filter our data using multiple conditions. SQL provides logical operators (AND, OR, NOT) and pattern matching to help us create more sophisticated queries.

## Using AND, OR, and NOT

The **AND** operator requires both conditions to be true:


In [ ]:
%%sql
SELECT name, age, home_planet
FROM student
WHERE age >= 18 AND midichlorian_count > 14000;

Running query in 'sqlite:///jedi_academy.db'

,name,age,home_planet
0,Luke Skywalker,19,Tatooine
1,Ben Solo,20,Chandrila


This finds adult students with high midichlorian counts.

The **OR** operator requires at least one condition to be true:


In [ ]:
%%sql
SELECT name, home_planet
FROM student
WHERE home_planet = 'Tatooine' OR home_planet = 'Alderaan';

Running query in 'sqlite:///jedi_academy.db'

,name,home_planet
0,Luke Skywalker,Tatooine
1,Leia Organa,Alderaan


This shows us students whose home planet is either Tatooine or Alderaan.

The **NOT** operator negates a condition:

In [ ]:
%%sql
SELECT name, age, home_planet
FROM student
WHERE NOT age < 18;

Running query in 'sqlite:///jedi_academy.db'

,name,age,home_planet
0,Luke Skywalker,19,Tatooine
1,Leia Organa,19,Alderaan
2,Ben Solo,20,Chandrila
3,Barriss Offee,18,Mirial


This finds all students who aren't younger than 18 (another way of saying age >= 18).

## Pattern Matching with LIKE

The **LIKE** operator lets you search for patterns in text. It uses two special characters:
- `%` matches any sequence of characters
- `_` matches any single character

Find all students whose names start with 'L':

In [ ]:
%%sql
SELECT name, home_planet
FROM student
WHERE name LIKE 'L%';

Running query in 'sqlite:///jedi_academy.db'

,name,home_planet
0,Luke Skywalker,Tatooine
1,Leia Organa,Alderaan


Find all students with five-letter home planets:

In [ ]:
%%sql
SELECT name, home_planet
FROM student
WHERE home_planet LIKE '_____';

Running query in 'sqlite:///jedi_academy.db'

,name,home_planet
0,Byph,Ithor
1,Ganodi,Rodia


## Using BETWEEN for Range Queries

The **BETWEEN** operator provides an easy way to check if a value falls within a range:

In [ ]:
%%sql
SELECT name, age, midichlorian_count
FROM student
WHERE midichlorian_count BETWEEN 13000 AND 14000;

Running query in 'sqlite:///jedi_academy.db'

,name,age,midichlorian_count
0,Leia Organa,19,13900
1,Barriss Offee,18,13200
2,Caleb Dume,14,13800
3,Gungi,12,13100


## Column Aliases with AS

Sometimes column names are unclear or we perform calculations that need better labels. The **AS** keyword lets us rename columns in our results:


In [ ]:
%%sql
SELECT
    name AS student_name,
    home_planet AS origin,
    midichlorian_count AS force_potential
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,student_name,origin,force_potential
0,Luke Skywalker,Tatooine,14500
1,Leia Organa,Alderaan,13900
2,Ben Solo,Chandrila,14800
3,Barriss Offee,Mirial,13200
4,Caleb Dume,Coruscant,13800
5,Petro,Corellia,12500
6,Katooni,Tholoth,12300
7,Gungi,Kashyyyk,13100
8,Zatt,Mon Cala,12200
9,Byph,Ithor,12100


This can help make our results more readable.

AS is especially useful when working with calculations:

In [ ]:
%%sql
SELECT
    name,
    age,
    midichlorian_count / 1000.0 AS force_rating
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,name,age,force_rating
0,Luke Skywalker,19,14.5
1,Leia Organa,19,13.9
2,Ben Solo,20,14.8
3,Barriss Offee,18,13.2
4,Caleb Dume,14,13.8
5,Petro,12,12.5
6,Katooni,11,12.3
7,Gungi,12,13.1
8,Zatt,11,12.2
9,Byph,11,12.1


## Aggregate Functions

Aggregate functions perform calculations across a set of rows. The basic aggregate functions are:
- **COUNT**: Counts rows
- **SUM**: Adds up values
- **AVG**: Calculates the average
- **MAX**: Finds the highest value
- **MIN**: Finds the lowest value

Let's see each in action:

### COUNT
Count how many students we have:

In [ ]:
%%sql
SELECT COUNT(*) AS total_students
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,total_students
0,11


### AVG and MAX

In [ ]:
%%sql
SELECT
    AVG(midichlorian_count) AS avg_force_potential,
    MAX(midichlorian_count) AS highest_potential
FROM student;

Running query in 'sqlite:///jedi_academy.db'

,avg_force_potential,highest_potential
0,13163.636364,14800


### Multiple Aggregates
We can combine multiple aggregates in one query:

In [ ]:
%%sql
SELECT
    COUNT(*) AS total_enrollments,
    AVG(grade) AS average_grade,
    MAX(grade) AS highest_grade,
    MIN(grade) AS lowest_grade
FROM enrollment
WHERE grade IS NOT NULL;

Running query in 'sqlite:///jedi_academy.db'

,total_enrollments,average_grade,highest_grade,lowest_grade
0,20,91.9,98.5,86.0


## Using WHERE with Aggregates

Important: The WHERE clause filters rows **before** aggregation occurs. For example:

In [ ]:
%%sql
SELECT AVG(midichlorian_count) AS avg_adult_potential
FROM student
WHERE age >= 18;

Running query in 'sqlite:///jedi_academy.db'

,avg_adult_potential
0,14100.0


# Combining Data: Understanding JOINs

One of the most powerful features of relational databases is the ability to combine data from multiple tables using **JOIN** operations. Remember how we split our data across different tables? Now we'll learn how to bring it back together.

## Why We Need JOINs

Let's say we want to see which master is teaching each course. We have:
- The master's name in the `master` table
- The course information in the `course` table
- A `master_id` in the `course` table connecting them

To see both together, we need to JOIN these tables.

## INNER JOIN

The most common type of join is the **INNER JOIN**. It connects rows from two tables based on a matching condition:

In [ ]:
%%sql
SELECT
    course.name AS course_name,
    master.name AS teacher_name
FROM course
INNER JOIN master ON course.master_id = master.master_id;

Running query in 'sqlite:///jedi_academy.db'

,course_name,teacher_name
0,Foundations of the Force,Yoda
1,Basic Lightsaber Training,Obi-Wan Kenobi
2,Advanced Combat Forms,Mace Windu
3,Meditation Techniques,Yoda
4,Advanced Force Control,Luminara Unduli
5,Force Healing Basics,Shaak Ti
6,Underwater Combat Training,Kit Fisto
7,Stealth and Infiltration,Quinlan Vos
8,Diplomatic Relations,Obi-Wan Kenobi
9,Force Precognition,Yoda


The `ON` clause specifies how the tables should be matched. Here, we're matching the `master_id` columns.

## LEFT JOIN

Sometimes we want to see all records from one table, even if they don't have matches in the other table. A **LEFT JOIN** does this:

In [ ]:
%%sql
SELECT
    student.name AS student_name,
    course.name AS course_name,
    enrollment.grade
FROM student
LEFT JOIN enrollment ON student.student_id = enrollment.student_id
LEFT JOIN course ON enrollment.course_id = course.course_id;

Running query in 'sqlite:///jedi_academy.db'

,student_name,course_name,grade
0,Luke Skywalker,Foundations of the Force,95.5
1,Luke Skywalker,Basic Lightsaber Training,88.0
2,Luke Skywalker,Advanced Combat Forms,91.5
3,Luke Skywalker,Meditation Techniques,89.0
4,Leia Organa,Foundations of the Force,92.5
5,Leia Organa,Meditation Techniques,94.0
6,Leia Organa,Force Healing Basics,98.5
7,Ben Solo,Foundations of the Force,96.0
8,Ben Solo,Advanced Force Control,88.5
9,Ben Solo,Diplomatic Relations,NaN


This shows all students, whether or not they're enrolled in any courses. Students without enrollments show NULL for course_name and grade.

## Multiple JOINs

We can join more than two tables. Let's see complete information about enrollments:


In [ ]:
%%sql
SELECT
    student.name AS student_name,
    course.name AS course_name,
    master.name AS teacher_name,
    enrollment.grade
FROM enrollment
JOIN student ON enrollment.student_id = student.student_id
JOIN course ON enrollment.course_id = course.course_id
JOIN master ON course.master_id = master.master_id
LIMIT 10;


Running query in 'sqlite:///jedi_academy.db'

,student_name,course_name,teacher_name,grade
0,Luke Skywalker,Foundations of the Force,Yoda,95.5
1,Luke Skywalker,Basic Lightsaber Training,Obi-Wan Kenobi,88.0
2,Luke Skywalker,Advanced Combat Forms,Mace Windu,91.5
3,Luke Skywalker,Meditation Techniques,Yoda,89.0
4,Leia Organa,Force Healing Basics,Shaak Ti,98.5
5,Leia Organa,Meditation Techniques,Yoda,94.0
6,Leia Organa,Foundations of the Force,Yoda,92.5
7,Ben Solo,Foundations of the Force,Yoda,96.0
8,Ben Solo,Advanced Force Control,Luminara Unduli,88.5
9,Ben Solo,Diplomatic Relations,Obi-Wan Kenobi,NaN


## Understanding GROUP BY

Now that we know how to combine data from multiple tables, let's learn how to analyze groups of related data. The **GROUP BY** clause lets us split our data into categories and calculate statistics for each category.

Let's start with a simple example: How many students is each master teaching?

In [ ]:
%%sql
SELECT
    master.name AS teacher_name,
    COUNT(DISTINCT student.student_id) AS number_of_students
FROM master
LEFT JOIN course ON master.master_id = course.master_id
LEFT JOIN enrollment ON course.course_id = enrollment.course_id
LEFT JOIN student ON enrollment.student_id = student.student_id
GROUP BY master.master_id, master.name;

Running query in 'sqlite:///jedi_academy.db'

,teacher_name,number_of_students
0,Yoda,8
1,Obi-Wan Kenobi,4
2,Mace Windu,1
3,Plo Koon,0
4,Shaak Ti,3
5,Kit Fisto,2
6,Luminara Unduli,1
7,Quinlan Vos,0


Note that:
- We use `COUNT(DISTINCT student.student_id)` to avoid counting students multiple times if they take multiple courses with the same master
- The LEFT JOINs ensure we see all masters, even those without students
- We GROUP BY `master_id` and `name` to ensure each master appears once

## Filtering Groups with HAVING

While WHERE filters individual rows before they're grouped, HAVING filters the groups themselves. For example, let's find masters whose students average above 90%:

In [ ]:
%%sql
SELECT
    master.name AS teacher_name,
    COUNT(enrollment.student_id) AS number_of_students,
    AVG(enrollment.grade) AS average_grade
FROM master
LEFT JOIN course ON master.master_id = course.master_id
LEFT JOIN enrollment ON course.course_id = enrollment.course_id
GROUP BY master.master_id, master.name
HAVING average_grade >= 90;


Running query in 'sqlite:///jedi_academy.db'

,teacher_name,number_of_students,average_grade
0,Yoda,11,91.409091
1,Obi-Wan Kenobi,4,92.666667
2,Mace Windu,1,91.500000
3,Shaak Ti,3,95.750000
4,Kit Fisto,2,91.500000


## Working with NULL Values

In [ ]:
%%sql
SELECT student.name, course.name
FROM enrollment
JOIN student ON enrollment.student_id = student.student_id
JOIN course ON enrollment.course_id = course.course_id
WHERE enrollment.grade IS NULL;

Running query in 'sqlite:///jedi_academy.db'

,name,name
0,Ben Solo,Diplomatic Relations
1,Katooni,Force Healing Basics


## Table: Basic SQL
| Category | SQL Command | Description |
| --- | --- | --- |
| SELECT | `SELECT c1, c2 FROM t` | Retrieves data from columns `c1` and `c2` in table `t` |
|  | `SELECT * FROM t` | Retrieves all columns from table `t` |
|  | `SELECT DISTINCT c1 FROM t` | Retrieves unique values from column `c1` in table `t` |
|  | `SELECT c1, aggregate_function(c2) FROM t` | Retrieves values from `c1` and applies an aggregate function (e.g., `SUM`, `AVG`, `COUNT`) to `c2` |
| WHERE | `SELECT * FROM t WHERE condition` | Retrieves rows from table `t` that satisfy the given `condition` |
|  | `SELECT * FROM t WHERE c1 LIKE pattern` | Retrieves rows from table `t` where values in column `c1` match the specified `pattern` (e.g., `'A%'` for values starting with 'A') |
|  | `SELECT * FROM t WHERE c1 IN (v1, v2, ...)` | Retrieves rows from table `t` where values in column `c1` are in the specified list of values |
|  | `SELECT * FROM t WHERE c1 BETWEEN v1 AND v2` | Retrieves rows from table `t` where values in column `c1` are within the specified range |
| ORDER BY | `SELECT * FROM t ORDER BY c1` | Retrieves all rows from table `t` and sorts them based on values in column `c1` in ascending order |
|  | `SELECT * FROM t ORDER BY c1 DESC` | Retrieves all rows from table `t` and sorts them based on values in column `c1` in descending order |
| LIMIT | `SELECT * FROM t LIMIT n` | Retrieves the first `n` rows from the result set of the `SELECT` statement |
|  | `SELECT * FROM t LIMIT n OFFSET m` | Skips the first `m` rows and retrieves the next `n` rows from the result set |
| GROUP BY | `SELECT c1, aggregate_function(c2) FROM t GROUP BY c1` | Groups rows based on values in column `c1` and applies the aggregate function to `c2` for each group |
| HAVING | `SELECT c1, aggregate_function(c2) FROM t GROUP BY c1 HAVING condition` | Retrieves grouped rows that satisfy the given `condition` based on the result of the aggregate function |
| JOIN | `SELECT * FROM t1 JOIN t2 ON t1.c1 = t2.c1` | Combines rows from tables `t1` and `t2` based on the matching values in column `c1` (inner join) |
|  | `SELECT * FROM t1 LEFT JOIN t2 ON t1.c1 = t2.c1` | Returns all rows from `t1` and the matching rows from `t2`, or `NULL` if no match is found (left join) |
| UNION | `SELECT c1 FROM t1 UNION SELECT c1 FROM t2` | Combines the result sets of two or more `SELECT` statements and removes duplicates |
|  | `SELECT c1 FROM t1 UNION ALL SELECT c1 FROM t2` | Combines the result sets of two or more `SELECT` statements without removing duplicates |
| Subquery | `SELECT * FROM t1 WHERE c1 IN (SELECT c1 FROM t2)` | Retrieves rows from `t1` where values in column `c1` exist in the result of the subquery |
|  | `SELECT * FROM t1 WHERE EXISTS (SELECT * FROM t2 WHERE t1.c1 = t2.c1)` | Retrieves rows from `t1` where the subquery returns at least one row |
| CASE | `SELECT c1, CASE WHEN condition THEN 'Value1' ELSE 'Value2' END FROM t` | Returns 'Value1' if the `condition` is true, otherwise 'Value2' for each row in the result set |
| CREATE TABLE | `CREATE TABLE t (c1 datatype, c2 datatype)` | Creates a new table named `t` with columns `c1` and `c2` of specified datatypes |
| DROP TABLE | `DROP TABLE t` | Deletes the table `t` from the database |


# Document Databases and JSON Data

While relational databases organize data in tables, **document databases** take a different approach: storing data in flexible, self-contained documents using JSON (JavaScript Object Notation). Modern SQL databases like SQLite also support JSON data, giving us the best of both worlds.

## Understanding JSON

JSON is a lightweight data format that's both human-readable and machine-friendly. It supports:
- Strings: `"Luke Skywalker"`
- Numbers: `19`
- Objects: `{"name": "Luke", "age": 19}`
- Arrays: `[1, 2, 3]`
- Boolean: `true` or `false`
- Null: `null`

Here's how our Jedi student data might look in JSON:

```javascript
{
    "name": "Luke Skywalker",
    "age": 19,
    "home_planet": "Tatooine",
    "midichlorian_count": 14500,
    "master": {
        "name": "Obi-Wan Kenobi",
        "rank": "Master"
    },
    "skills": ["Force Push", "Lightsaber Combat"],
    "enrollments": [
        {
            "course": "Foundations of the Force",
            "grade": 95.5,
            "completed": true
        },
        {
            "course": "Basic Lightsaber Training",
            "grade": 88.0,
            "completed": true
        }
    ]
}
```

## Using JSON in SQLite

SQLite lets us store and query JSON data using its JSON1 extension. Let's see how this works:

In [ ]:
%%sql
-- Create a table with a JSON column
CREATE TABLE student_profiles (
    student_id INTEGER PRIMARY KEY,
    profile JSON
);

-- Insert a student profile as JSON
INSERT INTO student_profiles (profile) VALUES (
    json_object(
        'name', 'Luke Skywalker',
        'age', 19,
        'home_planet', 'Tatooine',
        'skills', json_array('Force Push', 'Lightsaber Combat'),
        'master', json_object(
            'name', 'Obi-Wan Kenobi',
            'rank', 'Master'
        )
    )
);

Running query in 'sqlite:///jedi_academy.db'

1 rows affected.

""


We can query this JSON data using special JSON functions:

In [ ]:
%%sql
-- Extract a single value
SELECT json_extract(profile, '$.name') as student_name
FROM student_profiles;

Running query in 'sqlite:///jedi_academy.db'

,student_name
0,Luke Skywalker



## When to Use JSON?

JSON in databases is particularly useful for:

1. **Flexible Data Structures**
   ```javascript
   {
       "name": "Luke Skywalker",
       "force_abilities": ["Push", "Pull"],  // Luke has force abilities
       "lightsaber_color": "blue"
   }
   ```
   ```javascript
   {
       "name": "Han Solo",
       "weapons": ["DL-44 Blaster"],        // Han has weapons instead
       "ship": "Millennium Falcon"
   }
   ```

2. **Nested Information**
   ```javascript
   {
       "name": "Ahsoka Tano",
       "training_history": {
           "padawan_years": {
               "master": "Anakin Skywalker", // highly nested data
               "start_year": "21 BBY",
               "end_year": "19 BBY"
           },
           "post_order": {
               "aliases": ["Fulcrum"],
               "activities": ["Rebellion Support"]
           }
       }
   }
   ```

3. **Array Data**
   ```javascript
   {
       "name": "Obi-Wan Kenobi",
       "lightsaber_forms": [ // this is an array
           {"form": "Soresu", "mastery": "High"},
           {"form": "Ataru", "mastery": "Moderate"}
       ]
   }
   ```

   
### When to Use Which?

Use **Relational Tables** when:
- Data structure is consistent and well-defined
- You need complex queries across relationships
- Data integrity is crucial
- Multiple applications need to access the data

Use **JSON** when:
- Data structure varies between records
- You need to store nested or hierarchical data
- Schema flexibility is important
- You're storing document-like data (user profiles, game states, configurations)

Many modern applications use both approaches together, getting the best of both worlds!

## Practice Your SQL (Important!)
THe best way to learn SQL is to practice it! You can "run" the following cell to get some practice.

In [ ]:
!wget https://github.com/brendanpshea/colab-utilities/raw/main/sql_select_quiz.py -q -nc
from sql_select_quiz import sql_select_quiz_url
db_url = "https://github.com/brendanpshea/computing_concepts_python/raw/main/sql_select_quiz/jedi_academy.db"
json_url = "https://github.com/brendanpshea/computing_concepts_python/raw/main/sql_select_quiz/jedi_quiz.json"
sql_select_quiz_url(db_url, json_url)

## Review With Quizlet
Click the following cell to launch the quizlet review.

In [ ]:
%%html
<iframe src="https://quizlet.com/821793896/learn/embed?i=psvlh&x=1jj1" height="600" width="100%" style="border:0"></iframe>

## Glossary

| **Term** | **Definition** |
| --- | --- |
| "%pattern%" | Matches values that contain the specified pattern anywhere within the value. **Query:** `SELECT * FROM table_name WHERE column_name LIKE '___';` |
| "%pattern" | Matches values that end with the specified pattern. **Query:** `SELECT * FROM table_name WHERE column_name LIKE '___';` |
| "pattern%" | Matches values that begin with the specified pattern. **Query:** `SELECT * FROM table_name WHERE column_name LIKE '___';` |
| ACID | Stands for **Atomicity**, **Consistency**, **Isolation**, and **Durability**; a set of properties ensuring reliable transaction processing in databases. |
| AND | A logical operator that returns true if both conditions are true. **Query:** `SELECT * FROM table_name WHERE condition1 ___ condition2;` |
| AVG | A function that returns the average value of a numeric column. **Query:** `SELECT ___(column_name) FROM table_name;` |
| Column | A vertical division of data in a table, representing a specific attribute. |
| Conceptual-Logical Model | A high-level representation of data and the relationships among data entities, typically used in database design. |
| COUNT | A function that returns the number of rows that match a specified condition. **Query:** `SELECT ___(column_name) FROM table_name;` |
| CREATE TABLE | A SQL statement used to create a new table in a database. **Query:** `___ table_name (column1 datatype, column2 datatype, ...);` |
| DROP TABLE | A SQL statement used to delete an existing table and its data. **Query:** `___ table_name;` |
| Entity-Relationship Diagram (ERD) | A visual representation of entities and their relationships in a database. |
| Flat file | A simple database stored in a plain text file, with each line typically representing a single record. |
| Foreign Key | A field (or collection of fields) in one table that uniquely identifies a row in another table, creating a relationship between the two tables. |
| FROM | A SQL clause used to specify the table(s) from which to retrieve data. **Query:** `SELECT column_name(s) ___ table_name;` |
| GROUP BY | A SQL clause used to arrange identical data into groups. **Query:** `SELECT column_name(s) FROM table_name ___ column_name;` |
| HAVING | A SQL clause used to filter groups created by the GROUP BY clause. **Query:** `SELECT column_name(s) FROM table_name GROUP BY column_name ___ condition;` |
| IN | A logical operator that checks if a value is within a set of values. **Query:** `SELECT * FROM table_name WHERE column_name ___ (value1, value2, ...);` |
| INT | A data type used to store integer values. |
| LIKE | A SQL operator used to search for a specified pattern in a column. **Query:** `SELECT * FROM table_name WHERE column_name ___ 'pattern';` |
| NATURAL JOIN | A SQL join operation that automatically joins tables based on columns with the same name and compatible data types. **Query:** `SELECT * FROM table1 ___ table2;` |
| NOT | A logical operator that negates a condition. **Query:** `SELECT * FROM table_name WHERE ___ condition;` |
| OR | A logical operator that returns true if at least one of the conditions is true. **Query:** `SELECT * FROM table_name WHERE condition1 ___ condition2;` |
| ORDER BY | A SQL clause used to sort the result set by one or more columns. **Query:** `SELECT column_name(s) FROM table_name ___ column_name;` |
| Physical Model | A detailed representation of how data is stored in a database, including tables, columns, data types, and constraints. |
| Primary Key | A field (or collection of fields) in a table that uniquely identifies each row in that table. |
| Query | A request for data or information from a database. |
| REAL | A data type used to store floating-point numbers. |
| Relational database | A database structured to recognize relations among stored items of information. |
| SELECT | A SQL statement used to retrieve data from a database. **Query:** `___ column_name(s) FROM table_name;` |
| SELECT * | A SQL statement used to retrieve all columns from a table. **Query:** `___ FROM table_name;` |
| Structured Data | Data that is organized in a predefined manner, typically in tables with rows and columns. |
| Structured Query Language (SQL) | A standardized language for managing and manipulating relational databases. |
| SUM | A function that returns the sum of a numeric column. **Query:** `SELECT ___(column_name) FROM table_name;` |
| Table | A collection of related data held in a structured format within a database, consisting of rows and columns. |
| TEXT | A data type used to store non-numeric data, such as strings of characters. |
| Transaction | A sequence of database operations that are treated as a single unit, ensuring ACID properties. |
| Unstructured Data | Data that does not have a predefined data model or is not organized in a predefined manner. |
| VARCHAR | A data type used to store variable-length strings of characters. |
| WHERE | A SQL clause used to specify conditions for filtering rows in a query. **Query:** `SELECT column_name(s) FROM table_name ___ condition;` |